# Scrapy
- 웹사이트에서 데이터 수집을 위한 오픈소스 파이썬 프레임워크
- 멀티스레딩으로 데이터 수집
- daum news 데이터 수집

In [4]:
# install scrapy
# !pip install scrapy

In [6]:
import scrapy, requests
from scrapy.http import TextResponse

## 1. make project

In [8]:
!scrapy startproject news

New Scrapy project 'news', using template directory 'C:\Users\User\anaconda3\Lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\Desktop\code\news

You can start your first spider with:
    cd news
    scrapy genspider example example.com


In [14]:
!tree news /f

'tree'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [ ]:
!tree news

news
├── news
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       └── __init__.py
└── scrapy.cfg

3 directories, 7 files


- scrapy structure
    - items : 데이터의 모양 정의
    - middewares : 수집할때 header 정보와 같은 내용을 설정
    - pipelines : 데이터를 수집한 후에 코드를 실행
    - settings : robots.txt 규칙, 크롤링 시간 텀등을 설정
    - spiders : 크롤링 절차를 정의

## 2. xpath
- link, contents

In [18]:
url = 'https://news.daum.net/'
response = requests.get(url)
response = TextResponse(response.url, body=response.text, encoding='utf-8')
response

<200 https://news.daum.net/>

In [31]:
selector = '/html/body/div[2]/main/section/div/div[1]/div[1]/ul/li/div/div/strong/a/@href'
links = response.xpath(selector).extract()
len(links), links[:3]

(20,
 ['https://v.daum.net/v/20240923151053270',
  'https://v.daum.net/v/20240923150750118',
  'https://v.daum.net/v/20240923145528413'])

In [35]:
link = links[0]
response = requests.get(link)
response = TextResponse(response.url, body=response.text, encoding='utf-8')
response

<200 https://v.daum.net/v/20240923151053270>

In [43]:
title = response.xpath('//*[@id="mArticle"]/div[1]/h3/text()')[0].extract()
title

"전국 주택 2가구 중 1가구 이상은 '준공 20년 이상'"

### xpath
- html element 선택하는 방법
- scrapy 에서는 기본적으로 xpath를 사용
- syntax
    - // : 최상위 엘리먼트
    - \* : 모든 하위 엘리먼트 : css selector의 한칸띄우기와 같다.
    - [@id="value"] : 속성값 선택
    - / : 한단계 하위 엘리먼트 : css selector의 >와 같다.
    - [n] : nth-child(n)

## 3. items.py
- Data Model

In [ ]:
# %load news/news/items.py

In [47]:
%%writefile news/news/items.py
import scrapy

class NewsItem(scrapy.Item):
    title = scrapy.Field()
    link = scrapy.Field()

Overwriting news/news/items.py


## 4. spider.py
- wirte crawling process

In [55]:
%%writefile news/news/spiders/spider.py
import scrapy
from news.items import NewsItem

class NewsSpider(scrapy.Spider):
    name = 'news'
    allow_domain = ['daum.net']
    start_urls = ['https://news.daum.net']
    
    def parse(self, response):
        selector = '/html/body/div[2]/main/section/div/div[1]/div[1]/ul/li/div/div/strong/a/@href'
        links = response.xpath(selector).extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)

    def parse_content(self, response):
        item = NewsItem()
        item['link'] = response.url
        item['title'] = response.xpath('//*[@id="mArticle"]/div[1]/h3/text()')[0].extract()
        yield item

Overwriting news/news/spiders/spider.py


## 5. run scrapy
- news 디렉토리에서 아래의 커멘드 실행
- scrapy crawl news -o news.csv

In [66]:
%pwd

'C:\\Users\\User\\Desktop\\code'

In [1]:
import os

def my_tree(startpath, indent=''):
    items = sorted(os.listdir(startpath), key=lambda x: (os.path.isfile(os.path.join(startpath, x)), x), reverse=True)

    if indent == '':
        print('\033[1;32m' + os.path.basename(os.path.abspath(startpath)) + '\033[0m')

    for item in items:
        path = os.path.join(startpath, item)
        connector = '└── ' if item == items[-1] else '├── '

        if os.path.isdir(path):
            print(indent + connector + '\033[1;32m' + item + '\033[0m')
            my_tree(path, indent + ('    ' if item == items[-1] else '│   '))
        else:
            print(indent + connector + item)

# 이 부분에 원하는 경로를 넣으시면 됩니다.
my_tree('./news')

news
├── scrapy.cfg
├── news.csv
└── news
    ├── settings.py
    ├── pipelines.py
    ├── middlewares.py
    ├── items.py
    ├── __init__.py
    ├── spiders
    │   ├── spider.py
    │   ├── __init__.py
    │   └── __pycache__
    │       ├── spider.cpython-312.pyc
    │       └── __init__.cpython-312.pyc
    └── __pycache__
        ├── settings.cpython-312.pyc
        ├── items.cpython-312.pyc
        └── __init__.cpython-312.pyc
